In [78]:
import xmltodict
from bs4 import BeautifulSoup
import requests
response = requests.get('https://www.mwnation.com/feed/?paged=500')
doc = xmltodict.parse(response.text)
d = doc['rss']['channel']['item']
docs = list()
for i in d:
    #print(i)
    content = i['content:encoded']
    soup = BeautifulSoup(content, 'html.parser')
    docs.append(soup.get_text())
#print(docs)
from nltk.tokenize import word_tokenize
#return tokens
token_docs = [word_tokenize(i) for i in docs]
word_docs = list()
#remove punctuation
for token_doc in token_docs:
    word_docs.append([word for word in token_doc if word.isalpha()])
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
nostw_docs = list()
#remove stopwords
for word_doc in word_docs:
    nostw_docs.append([w for w in word_doc if not w in stop_words])
no1ltw_docs = list()
#remove 1 letter words and lowercase
for nostw_doc in nostw_docs:
    no1ltw_docs.append([w.lower() for w in nostw_doc if not len(w) <= 2])
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
lemma_docs = list()
for no1ltw_doc in no1ltw_docs:
    lemma_docs.append([lemmatizer.lemmatize(word) for word in no1ltw_doc])
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
stemmed_docs = list()
for lemma_doc in lemma_docs:
    stemmed_docs.append([stemmer.stem(word) for word in lemma_doc])
    
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

id2word = corpora.Dictionary(stemmed_docs)
corpus = [id2word.doc2bow(text) for text in stemmed_docs]
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,id2word=id2word,num_topics=10,chunksize = 100,update_every=1,
                                           passes=1000,alpha='auto',per_word_topics=True)
from pprint import pprint
pprint(lda_model.print_topics())
# Compute Perplexity
print('\nModel Perplexity: ', lda_model.log_perplexity(corpus))  #lower is better
coherence_model_lda = CoherenceModel(model=lda_model, texts=stemmed_docs, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nModel Coherence: ', coherence_lda)#higher is better

import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plotter
%matplotlib inline
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

[(0,
  '0.014*"artist" + 0.010*"special" + 0.010*"get" + 0.010*"music" + '
  '0.009*"said" + 0.009*"need" + 0.009*"nyasa" + 0.009*"festiv" + '
  '0.007*"perform" + 0.007*"big"'),
 (1,
  '0.029*"billion" + 0.018*"said" + 0.018*"pension" + 0.016*"fund" + '
  '0.016*"year" + 0.016*"remit" + 0.011*"the" + 0.011*"arrear" + 0.009*"due" + '
  '0.009*"firm"'),
 (2,
  '0.028*"music" + 0.022*"produc" + 0.017*"artist" + 0.015*"record" + '
  '0.013*"work" + 0.013*"countri" + 0.012*"howev" + 0.012*"ralph" + '
  '0.010*"malawi" + 0.008*"big"'),
 (3,
  '0.016*"movi" + 0.015*"said" + 0.015*"simba" + 0.012*"william" + '
  '0.011*"role" + 0.011*"the" + 0.011*"farmer" + 0.009*"group" + '
  '0.008*"ejiofor" + 0.008*"film"'),
 (4,
  '0.029*"percent" + 0.020*"said" + 0.018*"countri" + 0.015*"malawi" + '
  '0.014*"public" + 0.011*"zimbabw" + 0.011*"zambia" + 0.011*"govern" + '
  '0.011*"mozambiqu" + 0.011*"tanzania"'),
 (5,
  '0.021*"debat" + 0.014*"said" + 0.012*"candid" + 0.012*"trade" + '
  '0.011*"tfta" 

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.072347 -0.152787       1        1  16.525831
3     -0.010677  0.149859       2        1  14.836637
4      0.062591 -0.066114       3        1  14.358030
2     -0.182317 -0.024116       4        1  12.781588
0     -0.132819 -0.038291       5        1  12.274699
9      0.064978  0.092867       6        1   9.937987
1      0.018416 -0.037355       7        1   8.586631
6      0.087077  0.008578       8        1   5.433613
8      0.020970  0.058237       9        1   5.264695
7     -0.000567  0.009122      10        1   0.000291, topic_info=               Term       Freq      Total Category  logprob  loglift
348         percent  19.000000  19.000000  Default  30.0000  30.0000
1032          music  18.000000  18.000000  Default  29.0000  29.0000
286         billion   9.000000   9.000000  Default  28.0000  28.0000
686           debat  12.000000  12.000000  Default  27.0000  27.0000
1198         produc  10.000000  10.000000  Default  26.0000  26.0000
...             ...        ...        ...      ...      ...      ...
460   kaferapanjira   0.000007   3.038018  Topic10  -7.2903  -0.2262
461         kapwepw   0.000007   2.237803  Topic10  -7.2903   0.0795
462          launch   0.000007   1.437572  Topic10  -7.2903   0.5220
463           legal   0.000007   2.005870  Topic10  -7.2903   0.1889
464         lilongw   0.000007   4.193039  Topic10  -7.2903  -0.5485

[588 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
1338      5  0.285264       abus
1338      6  0.570529       abus
889       8  0.563664        acb
0         9  0.834722  accompani
890       8  0.853810      accus
...     ...       ...        ...
1378      6  0.487662     yoneco
1236      4  0.930535        you
887       3  0.987696     zambia
888       3  0.987696    zimbabw
1417      2  0.908183    zingeni

[819 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 4, 5, 3, 1, 10, 2, 7, 9, 8])